# Capstone: Detection of Products in a Shelf

## Imports

In [1]:
import os
import subprocess
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

## Paths

These are the paths to all the relevant folders.

In [2]:
tf_wsp_path = os.environ.get('CAPSTONE_PATH')+'/TensorFlow/workspace/training_products_shelf/'
annotations_path = tf_wsp_path+'annotations'
images_path = tf_wsp_path+'images'
config_path = tf_wsp_path+'models/my_ssd_resnet50_v1_fpn/pipeline.config'

## Label Map

In [3]:
label_ls = [{'name':'product1', 'id':1},
            {'name':'product2', 'id':2}]

Now we turn the list into the format that the object detection library requires.

In [4]:
with open(annotations_path+'\label_map.pbtxt', 'w') as f:
    for label in label_ls:
        f.write('item{\n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

First, we need to move to the folder that contains the script that generates the tf-records-files. To generate the train-data-records-file, we run the following code. Note: It took me quite a while to figure out how the slashes should look like.

In [7]:
slashed_images_path = images_path.replace('/', '\\')
slashed_annotations_path = annotations_path.replace('/', '\\')
subprocess.run('cd TensorFlow\scripts\preprocessing && python generate_tfrecord.py -x '+slashed_images_path+'\\train -l '+slashed_annotations_path+'\\label_map.pbtxt -o '+slashed_annotations_path+'\\train.record', shell=True)

CompletedProcess(args='cd TensorFlow\\scripts\\preprocessing && python generate_tfrecord.py -x C:\\Users\\selmig\\PycharmProjects\\Capstone\\TensorFlow\\workspace\\training_products_shelf\\images\\train -l C:\\Users\\selmig\\PycharmProjects\\Capstone\\TensorFlow\\workspace\\training_products_shelf\\annotations\\label_map.pbtxt -o C:\\Users\\selmig\\PycharmProjects\\Capstone\\TensorFlow\\workspace\\training_products_shelf\\annotations\\train.record', returncode=0)

The test-data-record-file is generated by running the following code.

In [8]:
subprocess.run('cd TensorFlow\scripts\preprocessing && python generate_tfrecord.py -x '+slashed_images_path+'\\test -l '+slashed_annotations_path+'\\label_map.pbtxt -o '+slashed_annotations_path+'\\test.record', shell=True)

CompletedProcess(args='cd TensorFlow\\scripts\\preprocessing && python generate_tfrecord.py -x C:\\Users\\selmig\\PycharmProjects\\Capstone\\TensorFlow\\workspace\\training_products_shelf\\images\\test -l C:\\Users\\selmig\\PycharmProjects\\Capstone\\TensorFlow\\workspace\\training_products_shelf\\annotations\\label_map.pbtxt -o C:\\Users\\selmig\\PycharmProjects\\Capstone\\TensorFlow\\workspace\\training_products_shelf\\annotations\\test.record', returncode=0)

## Pipeline Configuration

First, we download a pre-trained model from `https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md`. We extract everything into TensorFlow/workspace/training_products_shelf/pre-trained_models.

Now we copy the pipline.config into TensorFlow/workspace/training_products_shelf/models/my_ssd_resnet50_v1_fpn (or another folder name if we downloaded a different pre-trained model).

Finally, we change the num_classes, batch_size, fine_tune_checkpoint, fine_tune_checkpoint_type, (use_bfloat16), label_map_path, input_path, label_map_path, and input_path. We only need to change use_bfloat16 if we train on a TPU.

In [10]:
config = config_util.get_configs_from_pipeline_file(config_path)
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 640
       width: 640
     }
   }
   feature_extractor {
     type: "ssd_resnet50_v1_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 0.00039999998989515007
         }
       }
       initializer {
         truncated_normal_initializer {
           mean: 0.0
           stddev: 0.029999999329447746
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5


In [11]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(config_path, 'r') as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [12]:
pipeline_config.model.ssd.num_classes = 2 # number of different labels
pipeline_config.train_config.batch_size = 4 # larger batch sizes require more memory
pipeline_config.train_config.fine_tune_checkpoint = tf_wsp_path+'/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0' # path to checkpoint of pre-trained model
pipeline_config.train_config.fine_tune_checkpoint_type = 'detection'
pipeline_config.train_input_reader.label_map_path = annotations_path+'/label_map.pbtxt' # path to label map
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [annotations_path+'/train.record'] # path to training-tf-record-file
pipeline_config.eval_input_reader[0].label_map_path = annotations_path+'/label_map.pbtxt' # path to label map
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [annotations_path+'/test.record'] # path to test-tf-record-file

In [13]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(config_path, 'wb') as f:
    f.write(config_text)

## Model Training

First, we copy TensorFlow/models/research/object_detection/model_main_tf2.py into TensorFlow/workspace/training_products/shelf.

Second, we run 
> `python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config` in TensorFlow/workspace/training_products_shelf.